In [1]:
import pandas as pd
from calitp_data_analysis.tables import tbls
from siuba import _, collect, count, filter, show_query
from shared_utils.rt_dates import MONTH_DICT
from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS, YEAR, MONTH

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

### what is the grain of this dataset?
metrics values (upt/vrm/vrh/crm/voms) of a specifc **tos**, of specific **mode**, for a speicifc **agency** on a specific **year/month**

# Test reading in `dim_monthly_ntd_ridership_with_adjustments` from warehouse

In [3]:
monthly_ridership = (
    tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()
)

In [4]:
# reading in old monthly ridership parquet for comparison
old_march_2024_ridership = pd.read_parquet("gs://calitp-analytics-data/data-analyses/ntd/ntd_monthly_ridership_2024_March.parquet")

In [5]:
# what is in this table?
# all states
display(
    monthly_ridership.shape,
    # monthly_ridership.info,
    monthly_ridership.head(3),
)

(613184, 22)

,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,mode_type_of_service_status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms
0,"Glens Falls, NY",33598,2024-07-01,2024-07-01 21:44:31.635354+00:00,20120,2024,Small Systems Reporter,City of Glens Falls,Inactive,OR,None,Unknown,Other,DO,2120,2021-10,2021,10,0,0,0,0
1,"Glens Falls, NY",33598,2024-07-01,2024-07-01 21:44:31.635354+00:00,20120,2024,Small Systems Reporter,City of Glens Falls,Inactive,OR,None,Unknown,Other,DO,2120,2022-11,2022,11,0,0,0,0
2,"Glens Falls, NY",33598,2024-07-01,2024-07-01 21:44:31.635354+00:00,20120,2024,Small Systems Reporter,City of Glens Falls,Inactive,OR,None,Unknown,Other,DO,2120,2004-12,2004,12,0,0,0,0


In [6]:
print(monthly_ridership.dtypes)

uza_name                                    object
uace_cd                                     object
_dt                                         object
ts                             datetime64[ns, UTC]
ntd_id                                      object
year                                         int64
reporter_type                               object
agency                                      object
mode_type_of_service_status                 object
mode                                        object
mode_full_name                              object
service_type                                object
_3_mode                                     object
tos                                         object
legacy_ntd_id                               object
period_year_month                           object
period_year                                 object
period_month                                object
upt                                          int64
vrm                            

In [7]:
# are there NaN values in uza name? YES
monthly_ridership["uza_name"].isna().sum()

1608

In [8]:
# fill the NaN values with something
monthly_ridership["uza_name"].fillna("none", inplace=True)

In [9]:
# double check for na
monthly_ridership["uza_name"].isna().sum()

0

In [10]:
# can i now filter by uza CA, becaue there are zero NaN values? YES
uza_ca = monthly_ridership[monthly_ridership["uza_name"].str.contains(", CA")]
display(
    uza_ca.shape,
    uza_ca["uza_name"].value_counts()
)

(91924, 22)

Los Angeles--Long Beach--Anaheim, CA             19296
San Francisco--Oakland, CA                       10720
Sacramento, CA                                    8308
San Diego, CA                                     6164
Riverside--San Bernardino, CA                     3752
Stockton, CA                                      2948
Vallejo, CA                                       2144
Santa Rosa, CA                                    1876
Modesto, CA                                       1876
Lake Tahoe, CA-NV                                 1876
Seaside--Monterey--Pacific Grove, CA              1876
Palmdale--Lancaster, CA                           1608
San Luis Obispo, CA                               1608
Oxnard--San Buenaventura (Ventura), CA            1340
Visalia, CA                                       1340
Hanford, CA                                       1340
San Jose, CA                                      1340
Santa Cruz, CA                                    1340
Santa Barb

In [11]:
# where are the UTP values? the UTP page for this table used to be very wide
# look at a specific agency by ntd id, LA Metro == "9154"
# what is the dtype of ntd_id... its object, so use str.contrains
la_metro = monthly_ridership[monthly_ridership["ntd_id"] == "90154"]
display(la_metro.shape, uza_ca.shape)

(1876, 22)

(91924, 22)

In [12]:
# filter la_metro data to only see current and previous year (2024 and 2023)

la_2324 = la_metro[la_metro["period_year"].isin(["2023", "2024"])]

In [13]:
# does this 23/24 data match whats in the NTD report? YES
# confirm that data is now long instead of wide
la_2324[la_2324["period_year_month"] == "2024-03"][
    ["period_year_month", "mode", "tos", "upt"]
].sort_values(by="period_year_month", ascending=True)

,period_year_month,mode,tos,upt
38255,2024-03,VP,PT,125149
258756,2024-03,MB,PT,929537
258853,2024-03,MB,DO,18852906
286249,2024-03,HR,DO,1874721
295516,2024-03,LR,DO,3842909
588311,2024-03,DR,DO,62986
611829,2024-03,RB,DO,380625


# What to modify in the `produce_ntd_monthly_ridership_by_rtpa` function
step-by-step

In [14]:
# 1. reading in the data & renaming column
# intead of reading in the data via an excel doc by URL, intead will query the warehouse directly

#GOOD TO GO
full_upt = (tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()).rename(columns = {"mode_type_of_service_status": "Status"})


In [15]:
display(
    full_upt.dtypes,
)

uza_name                          object
uace_cd                           object
_dt                               object
ts                   datetime64[ns, UTC]
ntd_id                            object
year                               int64
reporter_type                     object
agency                            object
Status                            object
mode                              object
mode_full_name                    object
service_type                      object
_3_mode                           object
tos                               object
legacy_ntd_id                     object
period_year_month                 object
period_year                       object
period_month                      object
upt                                int64
vrm                                int64
vrh                                int64
voms                               int64
dtype: object

In [16]:
#1.1 setting `ntd_id` to str

# NTD_ID COL IS ALREADY OBJECT/STR DATA TYPE
# full_upt["ntd_id"] = full_upt["ntd_id"].astype(str)

In [17]:
#2 filter df to get rid of na rows
# GOOD TO GO!
full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)

In [18]:
full_upt["agency"].isna().sum()

0

In [19]:
#3 new DF of just CA and not-blank agency names
# filter UZA name col for CA and not blank agency names.

#GOOD TO GO!
ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)

In [20]:
display(
    ca.shape,
    ca.head()
)

(91924, 22)

,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,Status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms
0,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2005-12,2005,12,0,0,0,0
1,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2004-04,2004,4,0,0,0,0
2,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2011-10,2011,10,0,0,0,0
3,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2021-03,2021,3,0,0,0,0
4,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2015-10,2015,10,0,0,0,0


In [21]:
#4 read in ntd_id-to-RTPA crosswalk
# same as before, no changes

#GOOD TO GO!
crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
crosswalk.columns

Index(['ntd_id', 'Legacy NTD ID', 'Agency', 'UZA Name', 'RTPA_open_data',
       'RTPA'],
      dtype='object')

In [22]:
#5 merge CA df and crosswalk

# GOOD TO GO
df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        #change on = value to ntd_id
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
print(df._merge.value_counts())

both          91924
left_only         0
right_only        0
Name: _merge, dtype: int64


In [23]:
#6 flag to see if there are any unmerged rows 

# GOOD TO GO
if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")

In [24]:
#7 reverses the MONTH_DICT 

# Add new columns    
reversed_months = {v:k for k, v in MONTH_DICT.items()}
    
for m in range(1, reversed_months[month] + 1):
    df = add_change_columns(df, year, m)

NameError: name 'month' is not defined

In [25]:
# FINAL FORM


# final updated  `produce_ntd_monthly_ridership_by_rtpa` function.
# may not need the upt_url

def produce_ntd_monthly_ridership_by_rtpa(
    #upt_url: str,
    year: int,
    month: str
) -> pd.DataFrame:
    """
    Import NTD data from url, filter to CA, 
    merge in crosswalk, and save individual csvs.
    """
    full_upt = (tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()
               ).rename(columns = {"mode_type_of_service_status": "Status"})
    
    full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)
    #full_upt.to_parquet(
    #    f"{GCS_FILE_PATH}ntd_monthly_ridership_{year}_{month}.parquet"
    #)
    ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)
    
    crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
    df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        #change on = value to ntd_id
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
    print(df._merge.value_counts())
    
    # Good, everything merged, as we want
    if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
        
    # Add new columns    
    reversed_months = {v:k for k, v in MONTH_DICT.items()}
    
    for m in range(1, reversed_months[month] + 1):
        df = add_change_columns(df, year, m)
    
    df = df.assign(
        Mode_full = df.Mode.map(NTD_MODES),
        TOS_full = df.TOS.map(NTD_TOS)
    )
    
    return df


---
## Do I need `reserved_months` anymore?
RE: `produce_ntd_monthly_ridership_by_rtpa` function.


Since the new table uses a `yyyy-mm` format now. i dont think i need the revesre months anymore. what else needs to change?
reversed month was needed for the `add_change_columns` function.  



In [26]:
# what does reversed months do?

reversed_months = {v:k for k, v in MONTH_DICT.items()}

# what MONTH_DICT looks like initially.

print("this is MONTH_DICT"),
display(MONTH_DICT),

# like the name implies, reverses the key values pairs. 
print("this is reversed_months"),
display(reversed_months)


this is MONTH_DICT


{1: 'January',
 2: 'February',
 3: 'March',
 4: 'April',
 5: 'May',
 6: 'June',
 7: 'July',
 8: 'August',
 9: 'September',
 10: 'October',
 11: 'November',
 12: 'December'}

this is reversed_months


{'January': 1,
 'February': 2,
 'March': 3,
 'April': 4,
 'May': 5,
 'June': 6,
 'July': 7,
 'August': 8,
 'September': 9,
 'October': 10,
 'November': 11,
 'December': 12}

In [27]:
# what is the date format in the new df
# YYYY-MM
df["period_year_month"].sample(3)

76841    2023-10
29314    2011-09
38933    2019-10
Name: period_year_month, dtype: object

In [28]:
# what is the date format in the old data?
# M/YYYY
old_march_2024_ridership.iloc[:,18:22].head()

,9/2002,10/2002,11/2002,12/2002
0,132111.0,150074.0,127835.0,114857.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,32321.0,20384.0,13964.0,14050.0
4,5571149.0,6024277.0,5498884.0,5693491.0


## options
* change the df date format to match reversed_months format. (yyyy-mm to mm/yyy)
    * can use a mix of split, str replace, and reversing via slicing 
* change the `add_change_column` function to accept yyyy-mm
    * make a v2 of this function that accepts yyyy-mm, creates a new column, and applys some function by looking at the row period_year_month and finding the "period_year_month" -1 yr to calculate the difference.

# update `add_change_columns` function
need to update this function so it can work with the `produce_ntd_monthly_ridership_by_rtpa` function

In [29]:
df.dtypes

uza_name                          object
uace_cd                           object
_dt                               object
ts                   datetime64[ns, UTC]
ntd_id                            object
year                               int64
reporter_type                     object
agency                            object
Status                            object
mode                              object
mode_full_name                    object
service_type                      object
_3_mode                           object
tos                               object
legacy_ntd_id                     object
period_year_month                 object
period_year                       object
period_month                      object
upt                                int64
vrm                                int64
vrh                                int64
voms                               int64
RTPA                              object
_merge                          category
dtype: object

In [30]:
df.head()

,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,Status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms,RTPA,_merge
0,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2005-12,2005,12,0,0,0,0,San Joaquin Council of Governments,both
1,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2004-04,2004,4,0,0,0,0,San Joaquin Council of Governments,both
2,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2011-10,2011,10,0,0,0,0,San Joaquin Council of Governments,both
3,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2021-03,2021,3,0,0,0,0,San Joaquin Council of Governments,both
4,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2015-10,2015,10,0,0,0,0,San Joaquin Council of Governments,both


In [31]:
# updated to match new year-month format 

def add_change_columns_v2(
    df: pd.DataFrame,
    year: int,
    month: int
) -> pd.DataFrame:
    """
    updated to accept yyyy-mm format.
    
    """    
    ntd_month_col = f"{year}-{month}"
    prior_year_col = f"{int(year)-1}-{month}"
        
    df[f"change_1yr_{ntd_month_col}"] = df[ntd_month_col] - df[prior_year_col]
    df = get_percent_change(df, ntd_month_col, prior_year_col)
    
    return df

In [32]:
# can i use a lambda function to create a new column that:
# checks every row (x), if the row has a `period_year_month` value, does it have a `period_year_month` -1 year value in the same column? 
    # if true, then subtract `period_year_month` by `period_year_month`-1 year.
    # else return 0
year = 2023
month = "05"

current_y_m = f"{year}-{month}"
prior_y_m = f"{int(year)-1}-{month}"

# using current_y_m works and prior_year_col works
df[df["period_year_month"].str.contains(prior_y_m)].head(3)



,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,Status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms,RTPA,_merge
158,"Stockton, CA",85087,2024-07-01,2024-07-01 21:44:31.635354+00:00,90012,2024,Full Reporter,San Joaquin Regional Transit District,Inactive,VP,Vanpool,Demand Response,Bus,DO,9012,2022-05,2022,5,0,0,0,0,San Joaquin Council of Governments,both
343,"Los Angeles--Long Beach--Anaheim, CA",51445,2024-07-01,2024-07-01 21:44:31.635354+00:00,90036,2024,Full Reporter,Orange County Transportation Authority,Active,VP,Vanpool,Demand Response,Bus,PT,9036,2022-05,2022,5,21376,185282,5089,111,Orange County Transportation Authority,both
691,"Seaside--Monterey--Pacific Grove, CA",80362,2024-07-01,2024-07-01 21:44:31.635354+00:00,90062,2024,Full Reporter,Monterey-Salinas Transit,Active,VP,Vanpool,Demand Response,Bus,PT,9062,2022-05,2022,5,0,0,0,0,Transportation Agency for Monterey County,both


In [33]:
# first attempt at what im trying to do

# create a new column called "change 1yr"
# at each row in "period_month_year", do what? take the value of that row, then mimus the value of the previous period_mont_year -1

#this doesnt work becaues im not indicating how X is used, 
#df["change_1yr"] = df["period_year_month"].apply(lambda x: (df["period_year_month"].str.contains(current_y_m)["upt"]) - (df["period_year_month"].str.contains(prior_year_col)["upt"]))

#pseudo code
df["change_1yr"] = df["period_year_month"].apply(lambda x: (upt at x for current_y_m, agency, tos and mode)  - 
                                                 (upt at x for prior_y_m, current_y_m, agency, tos and mode))



SyntaxError: invalid syntax (4080223667.py, line 10)

In [34]:
# how can i see the specific UPT for a specific agency, mode & TOS and date?

#uses .loc at current_y_m UPT, position 0, minus .loc at previous_year_col UPT, position o
# but is is the same agency/mode/tos in both sides of the equation?
df.loc[df["period_year_month"] == current_y_m,"upt"].values[0] - df.loc[df["period_year_month"] == prior_y_m,"upt"].values[0]

0

In [43]:
df.loc[
        (df["period_year_month"] == current_y_m) &
        (df["ntd_id"] == df["ntd_id"]) &
        (df["mode"] == df["mode"]) &
        (df["tos"] == df["tos"]), "upt"
    ].head()

145          0
466      26683
576       5007
866       1740
1114    105433
Name: upt, dtype: int64

In [36]:
df.sample(3)

,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,Status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms,RTPA,_merge
84312,"Los Angeles--Long Beach--Anaheim, CA",51445,2024-07-01,2024-07-01 21:44:31.635354+00:00,90211,2024,Full Reporter,Anaheim Transportation Network,Active,DR,Demand Response,Demand Response,Bus,PT,9211,2018-07,2018,7,0,0,0,0,Orange County Transportation Authority,both
84930,"Petaluma, CA",68887,2024-07-01,2024-07-01 21:44:31.635354+00:00,90213,2024,Full Reporter,City of Petaluma,Active,DR,Demand Response,Demand Response,Bus,PT,9213,2019-03,2019,3,1548,5581,669,4,Metropolitan Transportation Commission,both
34441,"Sacramento, CA",77068,2024-07-01,2024-07-01 21:44:31.635354+00:00,90220,2024,Small Systems Reporter,City of Folsom,Inactive,MB,Motor Bus,Fixed Route,Bus,DO,9220,2013-11,2013,11,0,0,0,0,Sacramento Area Council of Governments,both


In [37]:
df.loc[df["period_year_month"] == prior_y_m, "upt"][:10]


158          0
343      21376
691          0
879       1602
1189     86705
1414     78357
1742     37289
1916    108774
2152       210
2486     11603
Name: upt, dtype: int64

In [38]:
def test_difference_func(row, current_y_m, prior_y_m ):
    current_ym_val =  df.loc[
        (df["period_year_month"] == current_y_m) &
        (df["ntd_id"] == row["ntd_id"]) &
        (df["mode"] == row["mode"]) &
        (df["tos"] == row["tos"]), "upt"
    ]
    
    prev_ym_val =  df.loc[
        (df["period_year_month"] == prior_y_m) &
        (df["ntd_id"] == row["ntd_id"]) &
        (df["mode"] == row["mode"]) &
        (df["tos"] == row["tos"]), "upt"
    ]
    
    return (current_ym_val - prev_ym_val)

In [39]:
test_df = df[:50].copy()

In [40]:
test_df.columns

Index(['uza_name', 'uace_cd', '_dt', 'ts', 'ntd_id', 'year', 'reporter_type',
       'agency', 'Status', 'mode', 'mode_full_name', 'service_type', '_3_mode',
       'tos', 'legacy_ntd_id', 'period_year_month', 'period_year',
       'period_month', 'upt', 'vrm', 'vrh', 'voms', 'RTPA', '_merge'],
      dtype='object')

In [41]:
year = 2023
month = "05"

current_y_m = f"{year}-{month}"
prior_y_m = f"{int(year)-1}-{month}"

display(
    current_y_m,
    prior_y_m
)

'2023-05'

'2022-05'

In [42]:
test_df["test_col"] = test_df.apply(lambda row: test_difference_func(row, current_y_m="2023-05", prior_y_m="2022-05"), axis=1)

ValueError: Cannot set a DataFrame with multiple columns to the single column test_col

So im having trouble with coming up with a `change 1yr` column, now that the dataset is long (instead of the previous wide)

what i was going for was something like:
- adding a new column to the dataset called `change 1yr`, that takes the `UPT` value of the `ntd_id, mod, tos` for current `period_year_month` (ex. 2024-05) row, and subtracts the `UPT` value from the previous year's `period_year_month` (ex. 2024-05) row of the same `ntd_id, mod, tos`.

- i thought i had something with a .apply(lambda x:) method, but wasnt etting anywhere.

any suggestions on how i should approach this?


---

In [ ]:
# what is the most recent period_year_month? as of 7/2/2024\
# should expect to see values up tp May 2024
display(
    ca[ca["period_year_month"] == "2024-04"].shape,
    # as of 7/2, zero rows for May 2024. but expect ntd to update that data soon.
    # 7/15, still no values for May 2024
    ca[ca["period_year_month"] == "2024-05"].shape
)

# Tiffany method to get change 1yr working

[see link to her old GH sctipt](https://github.com/CityOfLosAngeles/covid19-indicators/blob/master/processing_utils/utils.py#L306-L315)

The .shift() method in Pandas is used to shift the values of a DataFrame or Series by a specified number of periods along the given axis.
When you use .shift(), the data in the DataFrame or Series is moved up or down, while the index/columns stay in place. can move the entire dataframe up/down/left/right, or just a row or column in any direction
```
sort_cols = ["county", "state", "fips", "date"]
group_cols = ["county", "state", "fips"]

df = df.assign(
        previous_day = (df.sort_values(sort_cols)
                        .groupby(group_cols)["new_cases"]
                        .apply(lambda x: x.shift(1))
                       ),
        post_day = (df.sort_values(sort_cols)
                    .groupby(group_cols)["new_cases"]
                    .apply(lambda x: x.shift(-1))
                   ),
    )
```

In [44]:
# df of just riverside county transpo agency in select years
rcta = ca[(ca["ntd_id"].str.contains("90218")) & (ca["period_year"].isin(["2024","2023", "2022", "2021"]))]

In [45]:
display(
    rcta.columns,
    rcta.shape
)

Index(['uza_name', 'uace_cd', '_dt', 'ts', 'ntd_id', 'year', 'reporter_type',
       'agency', 'Status', 'mode', 'mode_full_name', 'service_type', '_3_mode',
       'tos', 'legacy_ntd_id', 'period_year_month', 'period_year',
       'period_month', 'upt', 'vrm', 'vrh', 'voms'],
      dtype='object')

(40, 22)

In [46]:
# list of column names to sort the df by
# lets sort by mode, tos, period_year_month
sort_cols = ["mode", "tos", "period_year_month"]
group_cols = ["mode", "tos"]

current_y_m = f"{year}-{month}"
prior_y_m = f"{int(year)-1}-{month}"

In [47]:
previous_day = (rcta.sort_values(sort_cols) #sort df by oldest date
                        .groupby(group_cols)["upt"] #returns only upt values
                        .apply(lambda x: x.shift(1)) #shifts upt values down by 1 step
                       )

display(
    type(previous_day),
    previous_day.head(),
    rcta.sort_values(sort_cols).head(5),
    rcta.sort_values(sort_cols).groupby(group_cols).head(5)
)
# result is a series of the "upt" col shifted down 1 place

/tmp/ipykernel_1010/1216189095.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  previous_day = (rcta.sort_values(sort_cols)


pandas.core.series.Series

2948       NaN
3209    4520.0
3102    4687.0
3009    5208.0
3200    5216.0
Name: upt, dtype: float64

,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,Status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms
2948,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-01,2021,1,4520,55123,1253,27
3209,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-02,2021,2,4687,55239,1300,29
3102,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-03,2021,3,5208,60660,1428,29
3009,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-04,2021,4,5216,60078,1397,29
3200,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-05,2021,5,4931,55171,1309,29


,uza_name,uace_cd,_dt,ts,ntd_id,year,reporter_type,agency,Status,mode,mode_full_name,service_type,_3_mode,tos,legacy_ntd_id,period_year_month,period_year,period_month,upt,vrm,vrh,voms
2948,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-01,2021,1,4520,55123,1253,27
3209,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-02,2021,2,4687,55239,1300,29
3102,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-03,2021,3,5208,60660,1428,29
3009,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-04,2021,4,5216,60078,1397,29
3200,"Riverside--San Bernardino, CA",75340,2024-07-01,2024-07-01 21:44:31.635354+00:00,90218,2024,Full Reporter,Riverside County Transportation Commission,Active,VP,Vanpool,Demand Response,Bus,PT,9218,2021-05,2021,5,4931,55171,1309,29


pandas.core.series.Series